<a href="https://colab.research.google.com/github/huavanson/CS114.K21/blob/master/sarcasm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"sonhuavan","key":"899cfa81c0f3f4051aee96eb7e74b10f"}'}

Sau khi up file kaggle.json lên thì tạo 1 thư mục có tên .kaggle đưa file vừa up vào 

In [65]:
!ls -lha kaggle.json
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

-rw-r--r-- 1 root root 65 Jun 22 03:29 kaggle.json


Bước này Download dataset trên kaggle về 


In [66]:
!kaggle datasets download -d rmisra/news-headlines-dataset-for-sarcasm-detection

news-headlines-dataset-for-sarcasm-detection.zip: Skipping, found more recently modified local copy (use --force to force download)


Check xem các thư viện cần thiết đã được install vào chưa 


In [ ]:
!pip install numpy
!pip install pandas 
!pip install sklearn

In [ ]:
import json
import pandas as pd, numpy as np, re, time
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [67]:
!unzip news-headlines-dataset-for-sarcasm-detection.zip

Archive:  news-headlines-dataset-for-sarcasm-detection.zip
replace Sarcasm_Headlines_Dataset.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Sarcasm_Headlines_Dataset.json  
replace Sarcasm_Headlines_Dataset_v2.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

Đọc dataset bằng Pandas và check số lượng data 

In [68]:
df=pd.read_json('Sarcasm_Headlines_Dataset.json',lines=True)
# shape and head
df.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


In [69]:
df.shape

(26709, 3)

check xem data có bị missing value hay không

In [70]:
print(df.isnull().any(axis = 0))

article_link    False
headline        False
is_sarcastic    False
dtype: bool


Column article_link không cần thiết để detect 1 headline có sarcasm hay không nên có thể được lượt bỏ 


In [71]:
df = df.drop(['article_link'], axis=1)

Dữ liệu không bị missing value nhưng cột tiêu đề nên được loại bỏ các kí tự không cần thiết vì vậy em sử dụng Regular Expression ( Biểu thức chính quy) để loại bỏ các kí tự đặc biệt đó ví dụ dấu nháy dấu chấm v.v

In [79]:
df['headline'] = df['headline'].apply(lambda s : re.sub('[^a-zA-Z]', ' ', s))


In [ ]:
df.head(5)

,headline,is_sarcastic
0,former versace store clerk sues over secret b...,0
1,the roseanne revival catches up to our thorn...,0
2,mom starting to fear son s web series closest ...,1
3,boehner just wants wife to listen not come up...,1
4,j k rowling wishes snape happy birthday in th...,0


In [80]:
features = df['headline']
labels = df['is_sarcastic']
features_compare=features


Xử dụng PorterStemmer để cắt giảm 1 từ thành từ gốc của nó 
Ví dụ : "reading" or "reader"
Sau khi stemming sẽ trở thành "read" ( cho cả 2 từ trên)

In [81]:
ps = PorterStemmer()
features = features.apply(lambda x: x.split())
features = features.apply(lambda x : ' '.join([ps.stem(word) for word in x]))

So sánh chuỗi trước và sau khi stemming 

In [82]:
print(features_compare[0])
print(features[0])

former versace store clerk sues over secret  black code  for minority shoppers
former versac store clerk sue over secret black code for minor shopper


tf-idf ( term frequency–inverse document frequency) hay còn gọi là tần số xuất hiện- tần số nghịch đảo của 1 từ trong 1 văn bản . Kỹ thuật này sử dụng để đếm số lần xuất hiện của 1 từ trong 1 văn bản nhiều hoặc ít để tìm ra những từ quan trọng trong 1 văn bản 

Link tham khảo tf-idf :https://en.wikipedia.org/wiki/Tf%E2%80%93idf

TfidfVectorizer dùng để chuyển đổi bộ dữ liệu sang 1 ma trận có các thông số tf-idf dưới đây là link thư viện scikitlearn


https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

In [83]:
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(max_features = 5000)
features = list(features)
features = tv.fit_transform(features).toarray()

features sau khi được chuyển sang ma trận 


In [84]:
print(features.shape)

(26709, 5000)


Vì dữ liệu khá nhiều nên em sẽ chia theo tỷ lệ 90-10

In [85]:
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size = 0.1, random_state = 0)

Dùng thử 4 model để train


---



---



In [87]:
# model 1:-
# linear support vector classifier
lsvc = LinearSVC()
# training model
lsvc.fit(features_train, labels_train)
# accuracy tập train và tập test
print(lsvc.score(features_train, labels_train))
print(lsvc.score(features_test, labels_test))   
# model 2:-
#  Gaussuan Naive Bayes
gnb = GaussianNB()
gnb.fit(features_train, labels_train)
print(gnb.score(features_train, labels_train))
print(gnb.score(features_test, labels_test))  
# model 3:-
# Logistic Regression
lr = LogisticRegression()
lr.fit(features_train, labels_train)
print(lr.score(features_train, labels_train)) 
print(lr.score(features_test, labels_test))     
# model 4:
# Random Forest Classifier
rfc = RandomForestClassifier(n_estimators = 10, random_state = 0)
rfc.fit(features_train, labels_train)
print(rfc.score(features_train, labels_train))
print(rfc.score(features_test, labels_test))    


0.9110574923038522
0.8360164732309996
0.7826358266078709
0.7162111568700861
0.8824777435726766
0.8375140396855111
0.9895998003161661
0.7963309621864471


Crawl Data trên trang https://www.theonion.com/latest để lấy những head line sarcasm

Install 1 số thư viện cần thiết

In [88]:
!pip install beautifulsoup4
! pip install requests

Hàm Crawl data sử dụng beautifulSoup 

In [92]:
def crawl(url, data):
  r1 = requests.get(url)
  coverpage = r1.content
  soup1 = BeautifulSoup(coverpage, 'html5lib')
  coverpage_news = soup1.find_all("h2", class_="sc-759qgu-0 cYlVdn cw4lnv-6 eXwNRE")
  for i in coverpage_news:
     data.append(i.get_text())

In [109]:
import requests
from bs4 import BeautifulSoup
url1 = "https://www.theonion.com/latest"
more_stories = ''
data= []
while len(data)<800:
  crawl(url, data)
  r1 = requests.get(url)
  coverpage = r1.content
  soup1 = BeautifulSoup(coverpage, 'html5lib')
  t = soup1.find_all("a", class_="sc-1out364-0 hMndXN js_link")
  more_stories = t[-1].get('href')
  url=url1+more_stories
  print(url)

https://www.theonion.com/latest?startTime=1584388020384
https://www.theonion.com/latest?startTime=1584115920131
https://www.theonion.com/latest?startTime=1583960880072
https://www.theonion.com/latest?startTime=1583848740360
https://www.theonion.com/latest?startTime=1583507820685
https://www.theonion.com/latest?startTime=1583349840518
https://www.theonion.com/latest?startTime=1583258400710
https://www.theonion.com/latest?startTime=1583152200832
https://www.theonion.com/latest?startTime=1582815600650
https://www.theonion.com/latest?startTime=1582734240096
https://www.theonion.com/latest?startTime=1582633800457
https://www.theonion.com/latest?startTime=1582315260793
https://www.theonion.com/latest?startTime=1582168620077
https://www.theonion.com/latest?startTime=1582051980524
https://www.theonion.com/latest?startTime=1581692940469
https://www.theonion.com/latest?startTime=1581543300169
https://www.theonion.com/latest?startTime=1581443700779
https://www.theonion.com/latest?startTime=158134

In [113]:
data

['Bucs Sign Replacement-Level Veteran To Hold Down QB Position Until They Draft Starter',
 'Frustrated Dog Has No Time To Jerk Off Now That Owner Home All Day',
 'Conservative Worries Relief Checks Would Discourage Americans From Providing For Selves By Killing Him And Taking His Property',
 '10 Most OMG Adorable Animals',
 'Timeline Of Trump Administration’s Response To Coronavirus',
 'Universal Pictures Makes Movies Currently In Theaters Available For Home Viewing Following Coronavirus Concerns',
 'Amazon Hires 100,000 New Workers To Cram In Close Quarters Just For Kicks',
 'Kansas Basketball Devastated They Won’t Get Opportunity To Vacate National Championship',
 'Fiat Recalls More Than 10,000 Cars For Not Looking Small And Weird Enough',
 'Nation’s 108 Million Service Industry Workers Assure Public That Job Is Just Way To Stay Busy After Winning Lottery Years Ago',
 'Who Said It: Kanye West Or An Instruction Manual For The Cuisinart CRC-400 Electric Rice Cooker?',
 'CDC Urges Ameri

In [111]:
data=list(dict.fromkeys(data))
print(len(data))

800


In [112]:
print(len(set(data)))

800
